In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from PIL import Image
import numpy as np
image = Image.open("img.png").convert('L')
img_np = np.array(image)
img_np.shape

(1025, 800)

In [3]:
from UNET import DoubleConv

x = torch.from_numpy(img_np).float()
x = torch.unsqueeze(x, dim=1).float()
x = x.permute(1,0,2).unsqueeze(0)
# print(x.shape)
doubleconv = DoubleConv(in_channels=1, out_channels=64)
doubleconv(x).shape
x.size()

torch.Size([1, 1, 1025, 800])

In [6]:
from UNET import UNET

model = UNET(n_channels=1, n_class=1)

In [8]:
output = model(x)

In [9]:
output.shape

torch.Size([1, 1, 1025, 800])